In [ ]:
from tqdm import tqdm
from numpy import asarray
import tensorflow as tf
import keras
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
#from keras.applications.resnet import ResNet50
#from keras.applications.resnet import ResNet50
from tensorflow.keras.applications.resnet import decode_predictions
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Input,UpSampling2D,Flatten,BatchNormalization,Dense,Dropout,GlobalAveragePooling2D
from tensorflow.keras import optimizers
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
from PIL import Image
import pandas as pd
import os

In [ ]:
#adam = Adam(lr=0.0001)

In [ ]:
IMAGE_SIZE = [512, 512]
train_path = '/kaggle/input/lung-cancer-dataset/Lung cancer dataset/train'
val_path = '/kaggle/input/lung-cancer-dataset/Lung cancer dataset/val'

folders = glob(train_path+'/*')
print("Number of classes:",len(folders))
num_classes = len(folders)
nb_epochs = 30

resnet=ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# Fixed for our dataset
NUM_CLASSES = num_classes

# Fixed for Cats & Dogs color images
CHANNELS = 3

IMAGE_RESIZE = 512
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'softmax'
OBJECTIVE_FUNCTION = 'categorical_crossentropy'

# Common accuracy metric for all outputs, but can use different metrics for different output
LOSS_METRICS = ['accuracy']

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
#NUM_EPOCHS = 60
#EARLY_STOP_PATIENCE = 3

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# Training images processed in each step would be no.-of-train-images / STEPS_PER_EPOCH_TRAINING
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 2

# These steps value should be proper FACTOR of no.-of-images in train & valid folders respectively
# NOTE that these BATCH* are for Keras ImageDataGenerator batching to fill epoch step input
BATCH_SIZE_TRAINING =128
BATCH_SIZE_VALIDATION =64

from tensorflow.keras.models import Model,Sequential
model4 = Sequential()

# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
model4.add(ResNet50(include_top = False, pooling = RESNET50_POOLING_AVERAGE, weights = 'imagenet'))

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model4.add(Dense(num_classes, activation = 'softmax'))

# Say not to train first layer (ResNet) model as it is already trained
model4.layers[0].trainable = False

model4.summary()

from tensorflow.keras import optimizers

sgd = optimizers.SGD(learning_rate = 0.0001, momentum = 0.9, nesterov = True)
adam = tf.keras.optimizers.Adam(lr=0.0001,
                                 name='Adam')
model4.compile(optimizer = adam, loss = OBJECTIVE_FUNCTION, metrics = LOSS_METRICS)





from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = IMAGE_RESIZE

data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = data_generator.flow_from_directory(
        train_path,
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE_TRAINING,
        class_mode='categorical')


validation_generator = data_generator.flow_from_directory(
        val_path,
        target_size=(image_size, image_size),
        batch_size=BATCH_SIZE_VALIDATION,
        class_mode='categorical')

history = model4.fit(
        train_generator,
        epochs = nb_epochs,
        validation_data=validation_generator#,
    #steps_per_epoch=STEPS_PER_EPOCH_TRAINING,
    #validation_steps=STEPS_PER_EPOCH_VALIDATION
        
        )

model4.save("Adam_lr0.0001_trial_2_classification_using_resnet50.h5")
print("Saved model to disk")

df = pd.DataFrame(history.history)
df.to_csv('Adam_lr0.0001_trial_2_Resnet50history.csv',index = False)

# **Predication**

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

# Load the model from the H5 file
model = load_model('/kaggle/working/Adam_lr0.0001_trial_2_classification_using_resnet50.h5')

DIRECTORY= "/kaggle/input/lung-cancer-dataset/Lung cancer dataset/test"
class_names=['Bengin case','Malignant case','Normal case']

data = {'class': [],
        'predicate_class': [],
        'status': []}
pcount=0
fcount=0
totalb=totalm=totaln=fb=fm=fn=0.0
df = pd.DataFrame(data)
for img in os.listdir(DIRECTORY):
    img_path=os.path.join(DIRECTORY,img)
    img_array=cv2.imread(img_path)
    img_array=cv2.resize(img_array,(512,512))
    img_array_=img_array
    img_array_=np.expand_dims(img_array_,axis=0)
    pred=model.predict(np.array(img_array_))
    output_class=class_names[np.argmax(pred)]
    s=img.split()
    ctemp=s[0]+" "+s[1]
    if(ctemp=='Bengin case'):
        totalb=totalb+1
    if(ctemp=='Malignant case'):
        totalm=totalm+1
    if(ctemp=='Normal case'):
        totaln=totaln+1
    if(ctemp==output_class):
        temp = {'class': ctemp, 'predicate_class': output_class , 'status':'pass'}
        pcount=pcount+1
        df = df.append(temp, ignore_index=True)
    else :
        temp = {'class': ctemp, 'predicate_class': output_class , 'status':'fail'}
        fcount=fcount+1
        df = df.append(temp, ignore_index=True)

        if(ctemp=='Bengin case'):
            fb=fb+1
        if(ctemp=='Malignant case'):
            fm=fm+1
        if(ctemp=='Normal case'):
            fn=fn+1
print("Predication Accuracy of each class")
print('Bengin case',(((totalb-fb)/totalb)*100))
print('Malignant case',(((totalm-fm)/totalm)*100))
print('Normal case',(((totaln-fn)/totaln)*100))
print("Predication Accuracy of the model is ",((((totalb-fb)/totalb)*100)+(((totalm-fm)/totalm)*100)+(((totaln-fn)/totaln)*100))/3)
print("Pass count=",pcount)
print("Fail count=",fcount)
df

In [ ]:
df.to_csv('Adam_data_lr0.0001.csv', index=False)

In [ ]:
!pwd

In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink(r'Adam_data_lr0.0001.csv')

In [ ]:
FileLink(r'Adam_lr0.0001_trial_2_Resnet50history.csv')

In [ ]:
FileLink(r'Adam_lr0.0001_trial_2_classification_using_resnet50.h5')